In [1]:
import os
import time
import copy
import torch
import random
import numpy as np
import matplotlib
#matplotlib.use('Agg')
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from PIL import Image as im
from model import Gauss2D

In /packages/parallel_studio_xe_2020/intelpython3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /packages/parallel_studio_xe_2020/intelpython3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /packages/parallel_studio_xe_2020/intelpython3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /packages/parallel_studio_xe_2020/intelpython3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will b

In [2]:
show        = True 
pretrain    = False
data_dir    = 'data'
filename    = 'model_out_gauss.pth'
batch_size  = 4       #Number of samples in each batch
num_workers = 4
epoch_num   = 200     #Number of epochs to train the network
lr          = 0.1     # Learning rate

In [3]:
data_transforms = {
    'train': transforms.ToTensor(),
    'val': transforms.ToTensor()
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=num_workers)
               for x in ['train', 'val']}

In [4]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) 

cuda


In [5]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

Gauss2D.imshow(out,title=[class_names[x] for x in classes], fname='initial.jpg')
for x in classes: 
    print(class_names[x])


model_ft = models.resnet18(pretrained=pretrain)
if torch.cuda.device_count() > 1: 
    print("Using ", torch.cuda.device_count(),"gpus!")
    model_ft = nn.DataParallel(model_ft) #
    
model_ft=model_ft.to(device)

one
one
one
one
Using  2 gpus!


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
model_ft = Gauss2D.train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epoch_num, dataloaders=dataloaders, device=device,dataset_sizes=dataset_sizes)
torch.save(model_ft.state_dict(), filename)

visualize_model(model_ft,num_images=10,device=device,dataloaders=dataloaders,class_names=class_names)

Epoch 0/199
----------


/home/dennis/.local/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [ ]:
model_weights = 'model_out_gauss.pth'
model = Gauss2D()
model.load_state_dict(torch.load(model_weights,map_location=torch.device('cpu')))

In [31]:
dataList = []
for image in glob.glob('data/test/two'):
    data = Image.open(image)
    dataList.append(np.asarray(data))

In [34]:
print(model)

OrderedDict([('module.conv1.weight', tensor([[[[ 1.8332e-02,  5.9477e-02, -9.6548e-03,  ...,  5.8315e-02,
            4.2350e-02,  1.5458e-02],
          [ 4.6591e-03, -7.8863e-04,  3.9733e-02,  ...,  5.0721e-02,
           -1.2985e-02, -3.9677e-03],
          [ 5.2637e-02,  6.9286e-03,  2.1472e-02,  ..., -3.4765e-03,
            2.3531e-02,  6.2295e-04],
          ...,
          [ 1.0299e-02,  4.6616e-02,  1.7387e-02,  ...,  2.7320e-02,
            6.6667e-02,  3.2868e-02],
          [ 7.0379e-02,  1.7004e-02,  2.1333e-03,  ...,  1.2370e-02,
            3.1246e-02,  1.8798e-02],
          [ 2.3108e-02,  4.2725e-02,  7.9977e-02,  ...,  4.3635e-02,
           -1.3531e-02,  4.0762e-02]],

         [[ 2.3806e-02, -2.8121e-02,  3.1253e-02,  ...,  1.5383e-02,
            4.4150e-02,  6.2479e-02],
          [-8.3559e-03, -4.3841e-04,  1.5303e-02,  ...,  6.2724e-02,
           -4.1539e-03, -2.2360e-03],
          [-2.6559e-02, -5.1595e-02,  1.7997e-02,  ...,  1.3150e-02,
           -6.4888e-0